In [4]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import urllib

In [5]:
with open('players.txt') as f:
    mylist = [tuple(map(str, i.split(','))) for i in f]

player_list = [(i[0].strip("'('"), i[1].strip(" )\n'")) for i in mylist]

In [6]:
# # reset player_list if scraping was interrupted.
# # last_player should equal the last player scraped. This will prevent repeated data.

# last_player = 'Carmelo Anthony'
# last_tup = [i for i in player_list if i[0] == 'Carmelo Anthony']

# player_list = [i for i in player_list if i[0] not in all_df.Player.unique()]

# player_list += last_tup

# all_df = all_df[all_df.Player != last_player]

In [7]:
x = 1
all_df = pd.DataFrame()
for player in player_list:
    reference_site = 'https://www.basketball-reference.com' + player[1]
    page = urllib.request.urlopen(reference_site)
    soup = BeautifulSoup(page, 'lxml')
    per_game_table = soup.table
    
        # BBR comments out a lot of the tables, and this function
        # pulls information from those tables
        # Taken from:
        # https://stackoverflow.com/questions/49766150/when-scraping-data-from-basketball-reference-how-come-certain-tables-are-comment
    all_list = []
    if per_game_table != None:

        for comment in soup.find_all(string=lambda text:isinstance(text,Comment)):
            data=BeautifulSoup(comment, "lxml")
            for items in data.select("table.row_summable tr"):
                tds = [item.get_text(strip=True) for item in items.select("th,td")]
                all_list.append(tds)
        # get index of table headers
        df_starts = []
        for i, row in enumerate(all_list):
            if row[0] == 'Season':
                df_starts.append(i)

        base = list(player)
        # create table of advanced stats
        table = [row for row in all_list[df_starts[2]:df_starts[3]]]

        # convert each table to a DataFrame and add to list of dataframes
        cols = ['Player', 'href'] + table[0]
        player_df = pd.DataFrame(table[1:], columns=table[0])
        player_df['Player'] = player[0]
        player_df['href'] = player[1]

        # drop rows with totals and incorrect values


        all_df = all_df.append(player_df)
    print(x, player[0])
    x += 1


1 Sim Bhullar
2 Brian Skinner
3 Diamond Stone


KeyboardInterrupt: 

In [14]:
# filter out all columns that are not single season like 
# career or team sums
regex = '[0-9]{4}-[0-9]{2}'
filt = all_df['Season'].str.contains(regex)

final_df = all_df[filt]

In [17]:
final_df.head()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,,ORtg,DRtg,Player,href
0,2016,22,HOU,NBA,PG,4,0,52,0.9,6.5,...,0.9,0.9,0.0,5.5,2.8,,73,114,Isaiah Taylor,/players/t/taylois01.html
1,2017,23,ATL,NBA,PG,67,9,1167,6.8,16.3,...,1.4,0.4,3.5,4.9,18.6,,101,114,Isaiah Taylor,/players/t/taylois01.html
2,2018,20,NOP,NBA,PG,61,16,1169,7.7,17.8,...,1.0,0.1,1.9,3.7,19.7,,102,117,Frank Jackson,/players/j/jacksfr01.html
3,2019,21,NOP,NBA,PG,51,1,656,7.0,18.1,...,1.1,0.3,3.0,4.7,20.1,,97,115,Frank Jackson,/players/j/jacksfr01.html
4,2007,22,WAS,NBA,C,35,0,320,7.4,21.0,...,1.0,0.8,2.9,5.0,21.0,,94,111,Oleksiy Pecherov,/players/p/pecheol01.html


In [18]:
final_df.reset_index(inplace=True, drop=True)

final_df['Season'] = final_df.Season.str[:4]

final_df.shape

/Users/drewadamski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(9193, 34)

In [19]:
final_df.head()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,,ORtg,DRtg,Player,href
0,2016,22,HOU,NBA,PG,4,0,52,0.9,6.5,...,0.9,0.9,0.0,5.5,2.8,,73,114,Isaiah Taylor,/players/t/taylois01.html
1,2017,23,ATL,NBA,PG,67,9,1167,6.8,16.3,...,1.4,0.4,3.5,4.9,18.6,,101,114,Isaiah Taylor,/players/t/taylois01.html
2,2018,20,NOP,NBA,PG,61,16,1169,7.7,17.8,...,1.0,0.1,1.9,3.7,19.7,,102,117,Frank Jackson,/players/j/jacksfr01.html
3,2019,21,NOP,NBA,PG,51,1,656,7.0,18.1,...,1.1,0.3,3.0,4.7,20.1,,97,115,Frank Jackson,/players/j/jacksfr01.html
4,2007,22,WAS,NBA,C,35,0,320,7.4,21.0,...,1.0,0.8,2.9,5.0,21.0,,94,111,Oleksiy Pecherov,/players/p/pecheol01.html


In [20]:
final_df.to_csv('Data/per100poss.csv')